In [37]:
import subprocess
import sys
import pkg_resources

required_packages = ["selenium==4.6.0", "webdriver-manager", "pandas"]

def install_packages():
    """
    Vérifie si les packages requis sont installés.
    Si ce n'est pas le cas, ils seront installés automatiquement.
    """
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])
    # Vérifie les packages installés
    installed_packages = {pkg.key for pkg in pkg_resources.working_set}
    
    # Vérifie les packages manquants
    missing_packages = [pkg for pkg in required_packages if pkg.split('==')[0] not in installed_packages]
    
    if missing_packages:
        print(f"Installation des dépendances manquantes : {', '.join(missing_packages)}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", *missing_packages])
    else:
        print("Tous les packages nécessaires sont déjà installés.")

# Appel de la fonction pour installer les packages nécessaires
install_packages()


import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.service import Service

import time

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service('/usr/local/bin/chromedriver')  # Remplacez par le chemin correct de ChromeDriver
browser = webdriver.Chrome(service=service, options=chrome_options)
print("Chromedriver lancé avec succès !")



def scraper(ingredient1: str, ingredient2: str, n: int):
    """
    explications

    """

    browser.get("https://www.marmiton.org/")
    time.sleep(5)

    try:
        cookies = browser.find_element(By.ID, "didomi-notice-agree-button")
        cookies.click()
        time.sleep(2)
        print("cookies acceptés")
    except NoSuchElementException:
        pass
    # Si le pop-up des cookies apparaît on les accepte (sinon il faudrait s'abonner), sinon on ne 
    # fait rien
    # Le temps de 5 secondes donne le temps aux cookies d'apparaître

    recherche = browser.find_element(By.ID, "mrtn-header-search-input")
    recherche.send_keys(ingredient1 + ingredient2)
    recherche.send_keys(Keys.RETURN)
    time.sleep(2)

    recettes = browser.find_elements(By.CSS_SELECTOR, ".recipe-card__title")  # Sélecteur pour les 
    # titres des recettes

    # Liste des titres des n premières recettes
    premieres_recettes = [recette.text for recette in recettes[:n]]

    #for i, recette in enumerate(premieres_recettes, start=1):
        #print(f"{i}. {recette}")

    liens = [lien.get_attribute("href") for lien in browser.find_elements(By.CSS_SELECTOR, ".recipe-card-link")]

    # Liste des n premiers liens correspondnat aux recettes proposées
    premiers_liens = [lien for lien in liens[:n]]

    #for i, lien in enumerate(premiers_liens, start=1):
       # print(f"{i}. {lien}")

    dico_recettes = {}
    for i in range(len(premiers_liens)):
        lien = premiers_liens[i]  #on prend le i ème lien de la liste de liens
        browser.get(lien)
        time.sleep(3)
        note = browser.find_element(By.CSS_SELECTOR, "span.recipe-header__rating-text")
        note = note.text
        nb_personnes = browser.find_element(By.CSS_SELECTOR, "input.recipe-ingredients__qt-counter__value.title-5")
        nb_personnes = nb_personnes.get_attribute("value")
        temps = browser.find_element(By.CSS_SELECTOR, "div.recipe-primary__item>span")
        temps = temps.text
        liste_ingredients = {}
        ingredients = browser.find_elements(By.CSS_SELECTOR, "div.card-ingredient")
        for ingredient in ingredients:
            ingr_nom = ingredient.get_attribute("data-name")
            try:
                quantite = ingredient.find_element(By.CSS_SELECTOR, "span.count").text
                #print("quantité", quantite)
            
            except:
                quantite = None  #si la quantité n'est pas indiquée, par exemple pour le sel ou le poivre

            try:
                unite = ingredient.find_element(By.CSS_SELECTOR, "span.unit").text
                # print("unité", unite)
            
            except:
                unite = None #s'il n'y a pas d'unité, pour les fruits ou légumes par exemple


            elements = {"quantité": quantite, "unité": unite}
            liste_ingredients[ingr_nom] = elements

            dico_recettes[premieres_recettes[i]] = liste_ingredients
    return dico_recettes

    #print(dico_recettes)

/tmp/ipykernel_133213/1688099005.py:3: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


Tous les packages nécessaires sont déjà installés.
Chromedriver lancé avec succès !


In [1]:
# Dans le terminal :
# wget https://storage.googleapis.com/chrome-for-testing-public/131.0.6778.87/linux64/chromedriver-linux64.zip
# unzip chromedriver-linux64.zip
# ls -l
# cd chromedriver-linux64
# sudo mv chromedriver /usr/local/bin/
# sudo chmod +x /usr/local/bin/chromedriver

!pip install selenium==4.6.0 # cette version permet de télécharger directement le bon driver
!pip install webdriver-manager==3.8.1

!sudo apt update
!sudo apt install -y libnss3 libx11-xcb1 libxcomposite1 libxcursor1 libxdamage1 libxi6 libxtst6 libxrandr2 libasound2 libatk1.0-0 libpangocairo-1.0-0 libgtk-3-0

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.service import Service

chrome_options = Options()
chrome_options.add_argument("--headless")  # sans interface graphique
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service('/usr/local/bin/chromedriver')
browser = webdriver.Chrome(service=service, options=chrome_options)
print("Chromedriver lancé avec succès !")

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:4 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]m       
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]      
Hit:6 https://ppa.launchpadcontent.net/git-core/ppa/ubuntu jammy InRelease     
Hit:7 https://apt.postgresql.org/pub/repos/apt jammy-pgdg InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,458 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,226 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,741 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,515 kB]
Get:12 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,224 B]
Fetched 8,328 kB in 2s (4,740 kB/s)         

In [2]:
import time
browser.get("https://www.marmiton.org/")
time.sleep(5)

try:
    cookies = browser.find_element(By.ID, "didomi-notice-agree-button")
    cookies.click()
    time.sleep(2)
    print("cookies acceptés")
except NoSuchElementException:
    pass
# Si le pop-up des cookies apparaît on les accepte (sinon il faudrait s'abonner), sinon on ne fait rien
# Le temps de 5 secondes donne le temps aux cookies d'apparaître


In [3]:
# Ingrédients : tomate et épinards

recherche = browser.find_element(By.ID, "mrtn-header-search-input")
recherche.send_keys("tomate+epinards")
recherche.send_keys(Keys.RETURN)
time.sleep(2)

In [4]:
# Résultats :

recettes = browser.find_elements(By.CSS_SELECTOR, ".recipe-card__title")  # Sélecteur pour les titres des recettes

# Nombre de résultats souhaités 
n = 1

# Liste des titres des premières recettes
premieres_recettes = [recette.text for recette in recettes[:n]]

for i, recette in enumerate(premieres_recettes, start=1):
    print(f"{i}. {recette}")


#####
liens = [lien.get_attribute("href") for lien in browser.find_elements(By.CSS_SELECTOR, ".recipe-card-link")]

# Liste des premiers liens correspondnat aux recettes proposées
premiers_liens = [lien for lien in liens[:n]]

for i, lien in enumerate(premiers_liens, start=1):
    print(f"{i}. {lien}")

1. Muffin quinoa épinard tomates séchées
1. https://www.marmiton.org/recettes/recette_muffin-quinoa-epinard-tomates-sechees_346962.aspx


In [5]:
# Dictionnaire: ingredients/quantites par recette
dico_recettes={}
for i in range (len(premiers_liens)):
    lien=premiers_liens[i] #on prend le i ème lien de la liste de liens
    browser.get(lien)
    time.sleep(3)
    note=browser.find_element(By.CSS_SELECTOR, "span.recipe-header__rating-text")
    note=note.text
    nb_personnes=browser.find_element(By.CSS_SELECTOR, "input.recipe-ingredients__qt-counter__value.title-5")
    nb_personnes = nb_personnes.get_attribute("value")
    temps=browser.find_element(By.CSS_SELECTOR, "div.recipe-primary__item>span")
    temps=temps.text
    liste_ingredients={}
    ingredients=browser.find_elements(By.CSS_SELECTOR, "div.card-ingredient")
    #print(ingredients)
    for ingredient in ingredients:
        ingr_nom=ingredient.get_attribute("data-name")
        #ingr_nom=ingredient
        #print("ingr_nom", ingr_nom)
        try :
            quantite=ingredient.find_element(By.CSS_SELECTOR, "span.count").text
            #print("quantité", quantite)
            
        except:
            quantite=None #si la quantité n'est pas indiquée, par exemple pour le sel ou le poivre

        try:
            unite=ingredient.find_element(By.CSS_SELECTOR, "span.unit").text
            #print("unité", unite)
            
        except:
            unite=None #s'il n'y a pas d'unité, pour les fruits ou légumes par exemple


        elements={"quantité":quantite, "unité":unite}
        liste_ingredients[ingr_nom]=elements

        dico_recettes[premieres_recettes[i]]=liste_ingredients

    #for ingredient in ingredients:
     #   print(f"Ingrédient: {ingredient['ingredient']}, Quantité: {ingredient['quantite']}")
print(dico_recettes)

{'Muffin quinoa épinard tomates séchées': {'sel': {'quantité': '', 'unité': ''}, 'poivre': {'quantité': '', 'unité': ''}, 'cumin en poudre': {'quantité': '0.5', 'unité': 'cuillère à café'}, "piment d'Espelette": {'quantité': '1', 'unité': 'cuillère à café'}, 'persil': {'quantité': '', 'unité': ''}, 'oignon': {'quantité': '1', 'unité': ''}, 'tomate séchée': {'quantité': '100', 'unité': 'g'}, 'oeuf': {'quantité': '2', 'unité': ''}, 'quinoa': {'quantité': '100', 'unité': 'g'}, 'épinards': {'quantité': '200', 'unité': 'g'}, 'Parmesan': {'quantité': '50', 'unité': 'g'}, 'ail': {'quantité': '1', 'unité': 'gousse'}, "huile d'olive": {'quantité': '1', 'unité': 'cuillère à soupe'}}}


In [6]:
for recette, ingredients in dico_recettes.items():
    print(f"{recette}:")
    for ingredients, infos in ingredients.items():
        print(f"{ingredients}:")
        for cle, valeur in infos.items():
            print (f"{cle}:{valeur}")
        print()
    print()
    print()

Muffin quinoa épinard tomates séchées:
sel:
quantité:
unité:

poivre:
quantité:
unité:

cumin en poudre:
quantité:0.5
unité:cuillère à café

piment d'Espelette:
quantité:1
unité:cuillère à café

persil:
quantité:
unité:

oignon:
quantité:1
unité:

tomate séchée:
quantité:100
unité:g

oeuf:
quantité:2
unité:

quinoa:
quantité:100
unité:g

épinards:
quantité:200
unité:g

Parmesan:
quantité:50
unité:g

ail:
quantité:1
unité:gousse

huile d'olive:
quantité:1
unité:cuillère à soupe





In [82]:
# Calcul des calories

import pandas as pd
import lecturesa as l
df_calories, df_svm = l.lecture_excel()

def calcul_calories(ingredient, poids):
    """Renvoie le nombre de calories associé en kcal"""
    selection = df_calories[df_calories['Nom'].str.contains(ingredient, case=False, na=False)]
    resultat = 0
    if selection.empty:
        for key in calories_moyennes:
            if key in ingredient.lower():
                resultat = calories_moyennes[key]
        if resultat == 0:
            print("Ingrédient introuvable : " + ingredient)
            return 0
    else:
        resultat=selection["Calories"].mean()
    calories=resultat*poids/100
    return(calories)

print(calcul_calories("Oignon", 30))
print(calcul_calories("oeuf",60))
print(calcul_calories('mayonnaise',20))
print(calcul_calories('pain',100))

17.465
114.80021052631578
140.0
224.33333333333337


In [84]:
# Poids moyen et calories moyennes

poids_moyens = {
    "yaourt nature": 100,
    "pain de mie": 20,
    "oeuf": 60,
    "carotte": 125,
    "tomate": 120,
    "tomate cerise": 10,
    "oignon": 100,
    "ail": 50,
    "gousse d'ail": 5,
    "avocat": 200,
    "champignon de paris": 50,
    "concombre": 200,
    "céleri-rave": 1000,
    "cougette": 200,
    "céleri branche": 30,
    "endive": 150,
    "fenouil": 300,
    "poireau": 150,
    "poivron rouge": 150,
    "poivron jaune": 150,
    "poivron vert": 150,
    "potiron": 1500,
    "aubergine": 240,
    "brocoli": 300,
    "chou-fleur": 1500,
    "chou-vert": 2000,
    "betterave rouge": 300,
    "échalotte": 25,
    "potimarron": 1800,
    "courge spaghetti": 1800,
    "panais": 250,
    "artichaut": 300,
    "navet": 200,
    "pomme de terre": 120,
    "patate douce": 200,
    "mangue": 400,
    "melon": 500,
    "orange": 200,
    "poire": 200,
    "pomme": 150,
    "pêche": 100,
    "figue": 50,
    "citron vert": 90,
    "rhubarbe": 80,
    "abricot": 55,
    "datte": 12,
    "pruneau": 15,
    "banane": 150,
    "ananas": 2000}

calories_moyennes = {
    "mayonnaise" : 700,
    "pain" : 265,
    "yaourt" : 60
}

In [ ]:
# Conversion

def conversion(ingredient, quantite, unite):
    """
    Permet de calculer la quantité en grammes d'un ingrédient dont la quantité est donnée dans une autre unité

    """
    if unite.strip() == '':
        if ingredient in poids_moyens:
            quantite = quantite*poids_moyens[ingredient]
            unite = 'g'
    
    if unite in ["pincées", "pincée"]:
        quantite = quantite * 0.4
        unite = 'g'
        # on considère qu'une pincée correspond à 0,4g
        # les ingrédients mesurés en pincées sont souvent le sel ou le poivre dont la masse par pincée peut être estimée à 0,4g
        

    if unite in ["cl", "ml"]:
        facteur = 0 # Facteur de conversion de ml en g
        if ingredient in ["crème", "crème liquide", "lait"]:
            facteur = 1 
        if ingredient in ["huile", "huile d'olive", "huile végétale", "huile de colza", "huile de tournesol","mayonnaise"]:
            facteur = 0.92
        if facteur > 0:
            if unite == "ml":
                quantite = quantite*facteur
                unite = "g"
            if unite == "cl":
                quantite = quantite*10*facteur
                unite = "g"

    if unite == 'cuillère à soupe':
        quantite = quantite * 15 # Une cuillère à soupe contient en moyenne 15g
        unite = 'g'
    
    if unite == 'cuillère à café':
        quantite = quantite * 5 # Une cuillère à café contient en moyenne 5g
        unite = 'g'
    
    return ingredient,quantite,unite

ingredient = 'oeuf'
quantite = 3
unite = ' '
ingredient,quantite,unite = conversion(ingredient,quantite,unite)
print(ingredient)
print(quantite)
print(unite)

oeuf
180
g


In [70]:
# Calories d'une recette
recette = {'sel': {'quantité': '2', 'unité': 'pincées'}, 'poivre': {'quantité': '1', 'unité': 'pincée'}}

def calcul_calories_recette(recette):
    """
    prend en argument le dictionnaire renvoyé par le scraper contenenat les ingrédients, leur quantité et unité

    """
    calories = 0
    for ingredient,info in recette.items():
        quantite,unite = info['quantité'], info['unité']
        if quantite.strip() == '':
            quantite = 0
        else:
            quantite = float(quantite)
        ingredient,quantite,unite = conversion(ingredient,quantite,unite)
        if unite == 'g':
            calories += calcul_calories(ingredient,quantite)
        else:
            print('Ingredient non pris en compte : ' + ingredient)
    return calories
    
print(calcul_calories_recette(recette))

3.4115360000000003


In [90]:
def proposition_recette(ingredient1, ingredient2, n):
    """"Retourne les n recettes de plus basses calories utilisant les ingrédients de la liste ingredient"""
    
    recettes = scraper(ingredient1, ingredient2, n)
    proposition = {}
    for nom,composition in recettes.items():
        print(composition)
        calories = calcul_calories_recette(composition)
        proposition[nom] = calories
    return proposition

print(proposition_recette('chocolats','',2))

{'farine': {'quantité': '1', 'unité': 'cuillère à soupe'}, 'beurre doux': {'quantité': '35', 'unité': 'g'}, 'sucre semoule': {'quantité': '80', 'unité': 'g'}, 'chocolat noir': {'quantité': '120', 'unité': 'g'}, 'oeuf': {'quantité': '3', 'unité': ''}}
Ingrédient introuvable : beurre doux
Ingrédient introuvable : sucre semoule
{'beurre': {'quantité': '80', 'unité': 'g'}, 'levure': {'quantité': '1', 'unité': 'cuillère à café'}, 'farine': {'quantité': '60', 'unité': 'g'}, 'sucre en poudre': {'quantité': '100', 'unité': 'g'}, 'chocolat pâtissier': {'quantité': '150', 'unité': 'g'}, 'oeuf': {'quantité': '3', 'unité': ''}, 'amandes en poudre': {'quantité': '50', 'unité': 'g'}}
Ingrédient introuvable : sucre en poudre
Ingrédient introuvable : chocolat pâtissier
Ingrédient introuvable : amandes en poudre
{'Ramequins fondants au chocolat': np.float64(1070.692298245614), 'Cake au chocolat et amandes en poudre': np.float64(993.4464286803966)}


In [86]:
def recette_moins_calorique(ingredient1,ingredient2):
    n = 2
    proposition = proposition_recette(ingredient1,ingredient2,n)
    cle_minimale = min(proposition, key = proposition.get)
    valeur_minimale = round(proposition[cle_minimale])
    print(f"La recette '{cle_minimale}' est la recette la moins calorique avec {valeur_minimale} kilocalories")

recette_moins_calorique('tomates','oeufs')

{'persil': {'quantité': '', 'unité': ''}, 'sel': {'quantité': '', 'unité': ''}, 'poivre': {'quantité': '', 'unité': ''}, 'tomate': {'quantité': '2', 'unité': ''}, 'pain de mie': {'quantité': '', 'unité': 'tranche'}, 'oeuf': {'quantité': '5', 'unité': ''}, 'yaourt nature': {'quantité': '1', 'unité': ''}}
Ingredient non pris en compte : persil
Ingredient non pris en compte : sel
Ingredient non pris en compte : poivre
Ingredient non pris en compte : pain de mie
{'tomate': {'quantité': '1', 'unité': ''}, 'curry': {'quantité': '', 'unité': ''}, 'oeuf': {'quantité': '1', 'unité': ''}, 'pain de mie': {'quantité': '1', 'unité': ''}, 'ketchup': {'quantité': '', 'unité': ''}, 'gruyère': {'quantité': '', 'unité': ''}}
Ingredient non pris en compte : curry
Ingredient non pris en compte : ketchup
Ingredient non pris en compte : gruyère
La recette 'Tartine tomates oeufs rapido' est la recette la moins calorique avec 241 kilocalories
